In [39]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamyearbyyearstats
import pandas as pd
import matplotlib.pyplot as plt
import requests
import numpy as np
from pprint import pprint

In [90]:
champ_stats_df = pd.read_csv('Resources/champs_stats.csv')
champ_stats_df.head()

,Year,Team,Points Against,Result
0,2020,Los Angeles Lakers,107.6,4-2
1,2019,Toronto Raptors,108.4,4-2
2,2018,Golden State Warriors,107.5,4-0
3,2017,Golden State Warriors,104.3,4-1
4,2016,Cleveland Cavaliers,98.3,4-3


In [101]:
nba_teams = teams.get_teams()
all_champs = champ_stats_df['Team']
team_ids = {}
year = 2020
for champ in all_champs:
    team_ids[str(year)] = str([team for team in nba_teams if team["full_name"] == f'{champ}'][0]['id'])
    year = int(year) - 1
    

lakers_id = [team for team in nba_teams if team["full_name"] == "Los Angeles Lakers"][0]['id']
raptors_id = [team for team in nba_teams if team["full_name"] == "Toronto Raptors"][0]['id']
warriors_id = [team for team in nba_teams if team["full_name"] == "Golden State Warriors"][0]['id']
cavs_id = [team for team in nba_teams if team["full_name"] == "Cleveland Cavaliers"][0]['id']
spurs_id = [team for team in nba_teams if team["full_name"] == "San Antonio Spurs"][0]['id']
heat_id = [team for team in nba_teams if team["full_name"] == "Miami Heat"][0]['id']
mavs_id = [team for team in nba_teams if team["full_name"] == "Dallas Mavericks"][0]['id']
celtics_id = [team for team in nba_teams if team["full_name"] == "Boston Celtics"][0]['id']
pistons_id = [team for team in nba_teams if team["full_name"] == "Detroit Pistons"][0]['id']
team_ids
# print(lakers_id, raptors_id, warriors_id, cavs_id, spurs_id, heat_id, mavs_id, celtics_id, pistons_id)

{'2020': '1610612747',
 '2019': '1610612761',
 '2018': '1610612744',
 '2017': '1610612744',
 '2016': '1610612739',
 '2015': '1610612744',
 '2014': '1610612759',
 '2013': '1610612748',
 '2012': '1610612748',
 '2011': '1610612742',
 '2010': '1610612747',
 '2009': '1610612747',
 '2008': '1610612738',
 '2007': '1610612759',
 '2006': '1610612748',
 '2005': '1610612759',
 '2004': '1610612765',
 '2003': '1610612759',
 '2002': '1610612747',
 '2001': '1610612747',
 '2000': '1610612747'}

In [102]:
# Get team info for specific year
# team id is [0], city is [1], name is [2], Year is [3], GP is [4], wins is [5], win pct is [7], pts is [-2], lakers first on list is -5
full_team_info = {}
i = -5

for year, identity in team_ids.items():
    team_info = teamyearbyyearstats.TeamYearByYearStats(league_id='00', per_mode_simple='Totals', season_type_all_star='Regular Season', team_id=identity).get_dict()
    team_name = (team_info['resultSets'][0]['rowSet'][i:][0][1]) + ' ' + team_info['resultSets'][0]['rowSet'][i:][0][2]
    team_dict = {
        'Team': team_name,
        'ID': str(team_info['resultSets'][0]['rowSet'][i:][0][0]),
        'Games Played': team_info['resultSets'][0]['rowSet'][i:][0][4],
        'Win %': team_info['resultSets'][0]['rowSet'][i:][0][7],
        'Points': team_info['resultSets'][0]['rowSet'][i:][0][-2]
    }
    
    year_played = team_info['resultSets'][0]['rowSet'][i:][0][3]
    full_team_info[year_played] = team_dict
    i = i-1
    # pprint(team_info['resultSets'][0]['rowSet'][i:][0])

In [118]:
full_team_stats_df = pd.DataFrame(full_team_info, index=None).T.reset_index()
full_team_stats_df[['index', 'ID', 'Team', 'Win %', 'Points', 'Games Played']]
full_team_stats_df['Points Per Game'] = full_team_stats_df['Points'] / full_team_stats_df['Games Played']
full_team_stats_df = full_team_stats_df.rename(columns={'index': 'Year'})
full_team_stats_df.to_csv('Resources/regular_season_stats_champs.csv', index=False, header=True)

In [17]:
opponents_scores = [107.6, 108.4, 107.5, 104.3, 98.3, 99.9, 97.6, 95.0, 92.5, 96.0,
                    97.0, 99.3, 90.3, 90.1, 96.0, 88.4, 84.3, 90.4, 94.1, 97.2, 92.3]

C:\Users\K\AppData\Local\Temp\ipykernel_19932\2684625268.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  champs_points_df['Result'] = result
